# Verify effect of Hidden Similarity Regularization on Models

In [2]:
import pandas as pd

hidden_reg_df = pd.read_csv("../results/_hidden_reg.csv", delimiter=";")
hidden_reg_df

,algorithm,0.0_val,0.0_test,0.1_val,0.1_test,0.5_val,0.5_test
0,articulation_points,0.9953,0.2879,0.9901,0.1091,0.9956,0.9229
1,activity_selector,0.9849,0.7399,0.9802,0.9077,0.9787,0.9000
2,bellman_ford,0.9936,0.9609,0.9930,0.9785,0.9896,0.9063
3,bfs,1.0000,0.9941,1.0000,0.9980,1.0000,0.9946
4,binary_search,0.9912,0.4375,0.9922,0.4375,0.9922,0.4375
5,bridges,0.9902,0.8907,0.9970,0.8258,0.9970,0.8403
6,bubble_sort,0.9736,0.0898,0.9744,0.2354,0.9677,0.3945
7,dag_shortest_paths,0.9981,0.9937,0.9972,0.9824,0.9958,0.9834
8,dfs,0.9999,0.2524,0.9999,0.1367,0.9999,0.2705
9,dijkstra,0.9952,0.9795,0.9944,0.9912,0.9923,0.9741


# Load Datasets

In [3]:
import torch
import yaml
from algo_reasoning.src.sampler import CLRSDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/ronald/Documents/env/algo_reasoning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_algorithm_args(args_file):
    with open(args_file, 'r') as f:
        args = yaml.safe_load(f)

    return args

# Comparing effect of Hidden Regularization to Algorithm Length

In [5]:
algorithm_args = load_algorithm_args("../algorithm_args/default.yaml")
hidden_reg_df["max_length"] = pd.Series([0] * len(hidden_reg_df))

for alg in hidden_reg_df.algorithm:
    print("Generating sample for: ", alg)
    algorithms = [alg]
    nb_nodes = 64
    ds = CLRSDataset(algorithms, nb_nodes, 1, 1000, seed=7, algorithms_args=algorithm_args)
    obj = next(iter(ds)).to(device=device)

    hidden_reg_df.loc[hidden_reg_df.algorithm == alg, ["max_length"]] =  obj.max_length.item()

Generating sample for:  articulation_points
Generating sample for:  activity_selector
Generating sample for:  bellman_ford
Generating sample for:  bfs
Generating sample for:  binary_search
Generating sample for:  bridges
Generating sample for:  bubble_sort
Generating sample for:  dag_shortest_paths
Generating sample for:  dfs
Generating sample for:  dijkstra
Generating sample for:  find_maximum_subarray_kadane
Generating sample for:  floyd_warshall
Generating sample for:  graham_scan
Generating sample for:  heapsort
Generating sample for:  insertion_sort
Generating sample for:  kmp_matcher
Generating sample for:  lcs_length
Generating sample for:  matrix_chain_order
Generating sample for:  minimum
Generating sample for:  mst_kruskal
Generating sample for:  mst_prim
Generating sample for:  naive_string_matcher
Generating sample for:  optimal_bst
Generating sample for:  quickselect
Generating sample for:  quicksort
Generating sample for:  segments_intersect
Generating sample for:  strong

In [7]:
reg_1_effect = hidden_reg_df["0.1_test"] - hidden_reg_df["0.0_test"]
reg_5_effect = hidden_reg_df["0.5_test"] - hidden_reg_df["0.0_test"]

ood_gap_reg_0 = hidden_reg_df["0.0_val"] - hidden_reg_df["0.0_test"]
ood_gap_reg_1 = hidden_reg_df["0.1_val"] - hidden_reg_df["0.1_test"]
ood_gap_reg_5 = hidden_reg_df["0.5_val"] - hidden_reg_df["0.5_test"]

hidden_reg_df["reg_0.1_effect"] = reg_1_effect
hidden_reg_df["reg_0.5_effect"] = reg_5_effect
hidden_reg_df["ood_gap_0.0"] = ood_gap_reg_0
hidden_reg_df["ood_gap_0.1"] = ood_gap_reg_1
hidden_reg_df["ood_gap_0.5"] = ood_gap_reg_5

# Filtering algorithms to the ones that lack OOD Generalization
filtered_df = hidden_reg_df[hidden_reg_df["ood_gap_0.0"] > 0.1]

In [8]:
filtered_df

,algorithm,0.0_val,0.0_test,0.1_val,0.1_test,0.5_val,0.5_test,max_length,reg_0.1_effect,reg_0.5_effect,ood_gap_0.0,ood_gap_0.1,ood_gap_0.5
0,articulation_points,0.9953,0.2879,0.9901,0.1091,0.9956,0.9229,1041,-0.1788,0.6350,0.7074,0.8810,0.0727
1,activity_selector,0.9849,0.7399,0.9802,0.9077,0.9787,0.9000,65,0.1678,0.1601,0.2450,0.0725,0.0787
4,binary_search,0.9912,0.4375,0.9922,0.4375,0.9922,0.4375,7,0.0000,0.0000,0.5537,0.5547,0.5547
6,bubble_sort,0.9736,0.0898,0.9744,0.2354,0.9677,0.3945,2017,0.1456,0.3047,0.8838,0.7390,0.5732
8,dfs,0.9999,0.2524,0.9999,0.1367,0.9999,0.2705,192,-0.1157,0.0181,0.7475,0.8632,0.7294
10,find_maximum_subarray_kadane,0.9595,0.1406,0.9634,0.3281,0.9551,0.2656,64,0.1875,0.1250,0.8189,0.6353,0.6895
11,floyd_warshall,0.2673,0.1235,0.2724,0.1480,0.2661,0.1305,64,0.0245,0.0070,0.1438,0.1244,0.1356
13,heapsort,0.9736,0.3003,0.9863,0.4561,0.9905,0.1572,455,0.1558,-0.1431,0.6733,0.5302,0.8333
14,insertion_sort,0.9897,0.1118,0.9812,0.1313,0.9897,0.0923,64,0.0195,-0.0195,0.8779,0.8499,0.8974
15,kmp_matcher,0.9922,0.1563,0.9766,0.0938,0.9863,0.0938,19,-0.0625,-0.0625,0.8359,0.8828,0.8925


In [12]:
filtered_df[["reg_0.1_effect", "reg_0.5_effect"]].mean()

reg_0.1_effect    0.012884
reg_0.5_effect    0.034405
dtype: float64

In [13]:
filtered_df[["0.0_test", "0.1_test", "0.5_test"]].mean()

0.0_test    0.383242
0.1_test    0.396126
0.5_test    0.417647
dtype: float64

In [14]:
filtered_df[["ood_gap_0.0", "ood_gap_0.1", "ood_gap_0.5"]].mean()

ood_gap_0.0    0.522468
ood_gap_0.1    0.510247
ood_gap_0.5    0.487889
dtype: float64

In [15]:
_corr = hidden_reg_df[hidden_reg_df.columns.difference(['algorithm'])].corr()

_corr["max_length"]

0.0_test         -0.121707
0.0_val           0.167792
0.1_test         -0.107738
0.1_val           0.168622
0.5_test          0.090713
0.5_val           0.163092
max_length        1.000000
ood_gap_0.0       0.261021
ood_gap_0.1       0.240459
ood_gap_0.5       0.026399
reg_0.1_effect    0.052034
reg_0.5_effect    0.517309
Name: max_length, dtype: float64

# Effect by Algorithm Type

In [16]:
type_dict = {
    "divide_and_conquer": ["find_maximum_subarray_kadane"],
    "dynamic_programming": ["matrix_chain_order", "lcs_length", "optimal_bst"],
    "geometry": ["segments_intersect", "graham_scan", "jarvis_march"],
    "graphs": ["dfs", "bfs", "topological_sort", "articulation_points", "bridges", "strongly_connected_components", "mst_kruskal", "mst_prim", "bellman_ford", "dijkstra", "dag_shortest_paths", "floyd_warshall"],
    "greedy": ["activity_selector", "task_scheduling"], 
    "searching": ["minimum", "binary_search", "quickselect"],
    "sorting": ["insertion_sort", "bubble_sort", "heapsort", "quicksort"],
    "strings": ["naive_string_matcher", "kmp_matcher"]
}

def get_algo_type(algo):
    for _type in type_dict.keys():
        print
        if algo in type_dict[_type]:
            return _type
        
hidden_reg_df["_type"] = hidden_reg_df.algorithm.apply(get_algo_type)
agg_df = hidden_reg_df[hidden_reg_df.columns.difference(['algorithm'])].groupby(['_type']).mean()
agg_df

,0.0_test,0.0_val,0.1_test,0.1_val,0.5_test,0.5_val,max_length,ood_gap_0.0,ood_gap_0.1,ood_gap_0.5,reg_0.1_effect,reg_0.5_effect
_type,,,,,,,,,,,,
divide_and_conquer,0.140600,0.959500,0.328100,0.963400,0.265600,0.955100,64.0,0.818900,0.635300,0.689500,0.187500,0.125000
dynamic_programming,0.138500,0.211833,0.158667,0.226867,0.141500,0.222300,51.0,0.073333,0.068200,0.080800,0.020167,0.003000
geometry,0.985800,0.987250,0.951650,0.987000,0.988600,0.987300,62.5,0.001450,0.035350,-0.001300,-0.034150,0.002800
graphs,0.721483,0.932067,0.692950,0.932542,0.762092,0.930100,360.0,0.210583,0.239592,0.168008,-0.028533,0.040608
greedy,0.781100,0.990850,0.865000,0.988700,0.865600,0.987800,65.0,0.209750,0.123700,0.122200,0.083900,0.084500
searching,0.479167,0.987933,0.458333,0.994467,0.447933,0.993167,59.0,0.508767,0.536133,0.545233,-0.020833,-0.031233
sorting,0.172225,0.981925,0.227175,0.983175,0.178825,0.984900,720.5,0.809700,0.756000,0.806075,0.054950,0.006600
strings,0.281300,0.994150,0.265650,0.987300,0.125050,0.993150,13.5,0.712850,0.721650,0.868100,-0.015650,-0.156250


In [17]:
agg_df[["ood_gap_0.0", "ood_gap_0.1", "ood_gap_0.5"]]

,ood_gap_0.0,ood_gap_0.1,ood_gap_0.5
_type,,,
divide_and_conquer,0.818900,0.635300,0.689500
dynamic_programming,0.073333,0.068200,0.080800
geometry,0.001450,0.035350,-0.001300
graphs,0.210583,0.239592,0.168008
greedy,0.209750,0.123700,0.122200
searching,0.508767,0.536133,0.545233
sorting,0.809700,0.756000,0.806075
strings,0.712850,0.721650,0.868100


In [18]:
agg_df.corr()

,0.0_test,0.0_val,0.1_test,0.1_val,0.5_test,0.5_val,max_length,ood_gap_0.0,ood_gap_0.1,ood_gap_0.5,reg_0.1_effect,reg_0.5_effect
0.0_test,1.000000,0.393457,0.974702,0.391707,0.971587,0.392389,-0.177192,-0.669592,-0.634702,-0.666409,-0.446455,0.121266
0.0_val,0.393457,1.000000,0.439439,0.999894,0.369594,0.999925,0.180027,0.419367,0.433046,0.387237,0.048276,-0.005481
0.1_test,0.974702,0.439439,1.000000,0.438363,0.990840,0.436690,-0.190414,-0.607462,-0.619299,-0.652101,-0.235162,0.302025
0.1_val,0.391707,0.999894,0.438363,1.000000,0.369563,0.999892,0.181580,0.421009,0.434218,0.387242,0.051578,0.001309
0.5_test,0.971587,0.369594,0.990840,0.369563,1.000000,0.367168,-0.140346,-0.660815,-0.670219,-0.713536,-0.258293,0.352754
0.5_val,0.392389,0.999925,0.436690,0.999892,0.367168,1.000000,0.183244,0.420361,0.435800,0.389695,0.041914,-0.011432
max_length,-0.177192,0.180027,-0.190414,0.181580,-0.140346,0.183244,1.000000,0.320413,0.349461,0.276980,0.008227,0.111928
ood_gap_0.0,-0.669592,0.419367,-0.607462,0.421009,-0.660815,0.420361,0.320413,1.000000,0.976598,0.970899,0.479859,-0.124174
ood_gap_0.1,-0.634702,0.433046,-0.619299,0.434218,-0.670219,0.435800,0.349461,0.976598,1.000000,0.991839,0.280745,-0.301556
ood_gap_0.5,-0.666409,0.387237,-0.652101,0.387242,-0.713536,0.389695,0.276980,0.970899,0.991839,1.000000,0.287306,-0.357872


# Effect by Output Type

In [19]:
from algo_reasoning.src.specs import SPECS, Stage

def get_output_type(algo):
    for k, v in SPECS[algo].items():
        stage, _, _type = v

        if stage == Stage.OUTPUT:
            return _type
        
hidden_reg_df["_output_type"] = hidden_reg_df.algorithm.apply(get_output_type)
hidden_reg_df
output_agg_df = hidden_reg_df[hidden_reg_df.columns.difference(['algorithm', "_type"])].groupby(['_output_type']).mean()
output_agg_df

,0.0_test,0.0_val,0.1_test,0.1_val,0.5_test,0.5_val,max_length,ood_gap_0.0,ood_gap_0.1,ood_gap_0.5,reg_0.1_effect,reg_0.5_effect
_output_type,,,,,,,,,,,,
categorical,0.341400,0.380600,0.404000,0.426200,0.344000,0.412000,41.000000,0.039200,0.022200,0.068000,0.062600,0.002600
mask,0.787814,0.985857,0.776471,0.985386,0.904914,0.983771,514.285714,0.198043,0.208914,0.078857,-0.011343,0.117100
mask_one,0.356783,0.985267,0.372400,0.986900,0.309917,0.986817,44.666667,0.628483,0.614500,0.676900,0.015617,-0.046867
pointer,0.495993,0.828173,0.499607,0.828753,0.487660,0.827967,264.600000,0.332180,0.329147,0.340307,0.003613,-0.008333


In [20]:
output_agg_df[["ood_gap_0.0", "ood_gap_0.1", "ood_gap_0.5"]]

,ood_gap_0.0,ood_gap_0.1,ood_gap_0.5
_output_type,,,
categorical,0.039200,0.022200,0.068000
mask,0.198043,0.208914,0.078857
mask_one,0.628483,0.614500,0.676900
pointer,0.332180,0.329147,0.340307


In [21]:
output_agg_df[["0.0_test", "0.1_test", "0.5_test"]]

,0.0_test,0.1_test,0.5_test
_output_type,,,
categorical,0.341400,0.404000,0.344000
mask,0.787814,0.776471,0.904914
mask_one,0.356783,0.372400,0.309917
pointer,0.495993,0.499607,0.487660
